# Example 1: Basic Message Structure

In this example, we demostrate various ways to create a messages object.  The messages object is a list.  EAch entry in the list is a dictionary that represents a message containing keys suchs as 'role' and 'content'.  

For example, a typical message structure would look like this.

   ~~~
       messages = [
            {"role": "system", "content": "You are an expert in contract analysis."},
            {"role": "user", "content": "Please summarize the contract."}
       ]
   ~~~

## Messages Object - Various Approaches
The messages array is at the core of all Chat Completion APIs so it's very important that you undestand this.  When you leverage framrworks that interact with LLMs, often times they provide some sort of abstraction or wrapper for the raw structure.  [Here is a link](https://platform.openai.com/docs/api-reference/making-requests) to the OpenAI API reference documentation.  I highly recommend you first get familiar with how to call these endpoints by simply using CURL.

Below is an example of calling an OpenAI endpoint and what is important here is the messages array structure.  All these frameworks will end up pass the messages array in this format regardless of how they abstracte the messages array, which is very important.  When you are leveraging the Messages object in LangChain or LangGraph how you interact with it may be a little different but in the end your messages arrive at the endpoint in the format below.

   ~~~
        curl https://api.openai.com/v1/chat/completions \
        -H "Content-Type: application/json" \
        -H "Authorization: Bearer $OPENAI_API_KEY" \
        -d '{
            "model": "gpt-4o-mini",
            "messages": [{"role": "user", "content": "Say this is a test!"}],
            "temperature": 0.7
        }'
   ~~~

I want you to keep in mind what the messages array look like above and how we create a messages object in LangChain and be mindful that in the end, when the LLM receives your messages they **WILL** be in the format above!  Let's now take a look at the different ways we can constructure our messages array.

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pprint import pprint

# Define the system message and user message
contract_extraction_prompt = "You are an expert in contract analysis. Extract key clauses from the provided text."
full_text = "This contract includes a confidentiality clause, a termination clause, and a liability clause."

# Here is an example of how to create a messages
messages =  [
     {"role": "system", "content": contract_extraction_prompt },
     {"role": "user", "content": "This contract includes a confidentiality clause, a termination clause, and a liability clause."}
]

# Example 1 
print("\nExample 1:\n")
print("\nNotice in this example because we are not using the SystemMessage or HumanMessage classes so we cannot use pretty_print()\n")
for m in messages:
    pprint(m) # We have to use the pprint to pretty print each message


# Example 1a - same outcome, but different approach
print("\nExample 1a - same outcome but different approach:")
print("Notice how we are using the SystemMessage and HumanMessage classes to create the messages, these classes do have the pretty_print method defined.\n")
messages = [SystemMessage(content=f"You are an expert in contract analysis. Extract key clauses from the provided text.", name="Model")]
messages.append(HumanMessage(content=full_text,name="Rick"))

for m in messages:
    m.pretty_print()


# Example 1b - sample outcome, but different approach
print("\nExample 1b same outcome but different approach:")
messages = [{"role": "system", "content": contract_extraction_prompt}]
messages.append({"role": "user", "content": full_text})

for m in messages:
     print(m)

print("\nNow, let's print the messges using various ways as it's important to understand this strcuture:\n")
print("## pprint(messages) ##\n")
pprint(f"{messages} \n")
print("\n## print(messages) ##\n")
print(messages)
print("\n## pprint(messages[0] ##\n")
pprint(messages[0])


Example 1:


Notice in this example because we are not using the SystemMessage or HumanMessage classes so we cannot use pretty_print()

{'content': 'You are an expert in contract analysis. Extract key clauses from '
            'the provided text.',
 'role': 'system'}
{'content': 'This contract includes a confidentiality clause, a termination '
            'clause, and a liability clause.',
 'role': 'user'}

Example 1a - same outcome but different approach:
Notice how we are using the SystemMessage and HumanMessage classes to create the messages, these classes do have the pretty_print method defined.

================================ System Message ================================
Name: Model

You are an expert in contract analysis. Extract key clauses from the provided text.
================================ Human Message =================================
Name: Rick

This contract includes a confidentiality clause, a termination clause, and a liability clause.

Example 1b same outcome

# Example 2: Using a Message with a Custom Function Calling
Let's start out with the most basic function calling example in-which we *DO NOT* use an Agent or a Graph.  Technically, the only thing we are using LangChain for is the AzureCahtOpenAI class and the HumanMessage and FunctionMessage wrapper objects.

In [8]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, FunctionMessage
import json

# Load environment variables
root_dir = Path().absolute().parent
env_path = root_dir / '.env'
load_dotenv(dotenv_path=env_path)

# Initialize the model
model = AzureChatOpenAI(
    temperature=0,
    model_name="gpt-4o" # It's imporant to make sure you use the deployment name for the model you created in Azure, technically, you can call this anything!
)

# Define the function schema
functions = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    }
]

def get_weather(location: str, unit: str = "celsius") -> str:
    """Simulate getting weather data"""
    return f"It's 22°{'C' if unit == 'celsius' else 'F'} and sunny in {location}"

# Example conversation
messages = [
    HumanMessage(content="What's the weather like in London?")
]

# Get the model's response with function calling
response = model.invoke(messages, functions=functions)

# Check if the model wants to call a function
if response.additional_kwargs.get('function_call'):
    # Get function details
    function_call = response.additional_kwargs['function_call']
    function_name = function_call['name']
    function_args = json.loads(function_call['arguments'])
    
    # Call the function
    function_response = get_weather(**function_args)
    
    # Add the function result to the messages
    messages.append(response)
    messages.append(FunctionMessage(
        content=function_response,
        name=function_name
    ))
    
    # Get final response
    final_response = model.invoke(messages)
    print(final_response.content)

The weather in London is currently 22°C and sunny.


# Example 3: Using Prompts with Message History
This example shows how to maintain a conversation history by appending messages to the list.

In [ ]:
# Initialize message history
messages = []

# Add system message
messages.append({"role": "system", "content": "You are a helpful assistant."})

# User asks a question
messages.append({"role": "user", "content": "What are the main points of the contract?"})

# AI responds (simulated)
ai_response = "The main points include confidentiality, termination, and liability."

# Append AI response to message history
messages.append({"role": "assistant", "content": ai_response})

# Continue the conversation
messages.append({"role": "user", "content": "Can you explain the confidentiality clause?"})

print(json.dumps(messages, indent=2))

# Example 4: Using Prompt Templates
In this example, I show you various ways to use **Prompt Templates**.  This will have you better understand the various ways you can leverage prompt templates.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# Example 1: Basic PromptTemplate
basic_template = PromptTemplate.from_template(
    "You are a contract expert. Analyze the following text: {contract_text}"
)

# Example 2: ChatPromptTemplate with multiple messages
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a legal expert specialized in contract analysis."),
    ("human", "Please analyze this contract: {contract_text}"),
    ("assistant", "I'll analyze the contract focusing on key elements:"),
    ("human", "Also check for any {specific_clause} clauses.")
])

# Example 3: Template with structured formatting
structured_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are a legal expert with the following responsibilities:
    1. Analyze contract language
    2. Identify potential risks
    3. Suggest improvements"""),
    HumanMessage(content="Contract to analyze: {contract_text}")
])

# Sample contract text
contract_text = "This agreement includes confidentiality and non-compete clauses."
specific_clause = "termination"

# Format the templates
basic_formatted = basic_template.format(contract_text=contract_text)
chat_formatted = chat_template.format(
    contract_text=contract_text,
    specific_clause=specific_clause
)
structured_formatted = structured_template.format(
    contract_text=contract_text
)

# Print the results
print("Basic Template Result:")
print(basic_formatted)
print("\nChat Template Result:")
print(chat_formatted)
print("\nStructured Template Result:")
print(structured_formatted)

# Example of accessing individual messages from a chat template
print("\nAccessing Individual Messages from Chat Template:")
formatted_messages = chat_template.format_messages(
    contract_text=contract_text,
    specific_clause=specific_clause
)
for message in formatted_messages:
    print(f"\n{message.type}: {message.content}")

# Example 5: Streaming Messages
This is the most basic way to stream a response from the EndPoint.  Keep in mind taht we are not using LangGraph at the moment.  Our goal here is to build a strong foundation of understanding before diving deeper into LangGraph.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import os, getpass
from langchain_openai import AzureChatOpenAI

from dotenv import load_dotenv
import os
from pathlib import Path

# Get root directory path
root_dir = Path().absolute().parent.parent
env_path = root_dir / '.env'

# Load .env from root
load_dotenv(dotenv_path=env_path)
print(f"Loaded .env from {env_path}")
# Access variables
api_key = os.getenv('AZURE_OPENAI_API_KEY')

print(f"API Key: {  api_key[:4] + '*' * 28 + api_key[-4:] }")

# Create a message
msg = HumanMessage(content="Why is the Sky Blue?", name="Rick")
# messages = [HumanMessage(content=f"Why is the Sky blue?",name="Rick"))
messages = [msg]

llm = AzureChatOpenAI(model_name="gpt-4o")
for chunk in llm.stream(messages):
    # Check if we have content to print
    if chunk.content:
        print(chunk.content, end="", flush=True)


# Example 6: Streaming Messages using an Agent
Ok, so let's go **right** into the deep water, very quickly.  We having even dove into the concept of State objects or Agents yet, but we will cover those topics in future models.

For now, I wanted to show you a very simple example of a LangGraph Agent that makes use of a tool and one of the ways the response can be streamed.

In [13]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing import Type
from langchain.tools import tool
from pydantic import BaseModel

# Get root directory path
root_dir = Path().absolute().parent.parent
env_path = root_dir / '.env'

# Load .env from root
load_dotenv(dotenv_path=env_path)
print(f"Loaded .env from {env_path}")

# Access variables
api_key = os.getenv('AZURE_OPENAI_API_KEY')
print(f"API Key: {api_key[:4] + '*' * 28 + api_key[-4:]}")

# Define the input schema for our tool
class ClauseAnalysisInput(BaseModel):
    clause_type: str
    clause_text: str

# Create a custom tool for analyzing contract clauses
@tool(args_schema=ClauseAnalysisInput)
def analyze_clause(clause_type: str, clause_text: str) -> str:
    """
    Analyzes a specific contract clause and provides insights.
    
    Args:
        clause_type: The type of clause (e.g., 'confidentiality', 'termination', 'liability')
        clause_text: The actual text of the clause to analyze
    
    Returns:
        str: Analysis of the clause including key points and potential risks
    """
    analysis_template = {
        'confidentiality': lambda text: f"Confidentiality Analysis:\n- Scope: {text[:50]}...\n- Duration: Searching for time specifications...\n- Obligations: Identifying key responsibilities...",
        'termination': lambda text: f"Termination Analysis:\n- Notice Period: Searching...\n- Grounds for Termination: Analyzing conditions...\n- Post-termination obligations: Reviewing...",
        'liability': lambda text: f"Liability Analysis:\n- Limitation Scope: {text[:50]}...\n- Cap Amount: Searching for monetary limits...\n- Exclusions: Identifying carve-outs..."
    }
    
    if clause_type.lower() in analysis_template:
        return analysis_template[clause_type.lower()](clause_text)
    return f"Unrecognized clause type: {clause_type}. Please specify one of: confidentiality, termination, or liability."

# Initialize the model
model = AzureChatOpenAI(model_name="gpt-4o", temperature=0.5) # Example model initialization

# Add our custom tool to the tools list
tools = [analyze_clause]

# Define a dynamic system message
def state_modifier(state):
    return [
        SystemMessage(content="""You are a legal assistant specialized in contract analysis. 
        Use the analyze_clause tool to provide detailed insights about specific contract clauses.
        Always analyze each clause type mentioned in the user's request."""),
        *state["messages"]
    ]

# Create the agent with the state modifier and tools
langgraph_agent_executor = create_react_agent(model, tools, state_modifier=state_modifier)

# Prepare user message
user_message = (
    "Please analyze the following contract clauses in detail: "
    "1. Confidentiality Clause: All information shared during the project shall remain confidential for 5 years. "
    "2. Termination Clause: Either party may terminate with 30 days notice. "
    "3. Liability Clause: Total liability shall not exceed the total contract value. "
    "For each clause, provide a detailed analysis, including implications, potential risks, and any relevant legal considerations."
)

# Create a HumanMessage object
human_message = HumanMessage(content=user_message)

# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Stream the response - Sample example as earlier but using streaming

# Stream the response
print("Streaming response for the detailed analysis:")
for chunk in langgraph_agent_executor.stream({"messages": [human_message]}, config, stream_mode="updates"):
    # Check if the chunk contains agent messages
    if 'agent' in chunk:
        ai_messages = chunk['agent']['messages']
        print("AI Messages:")
        for ai_message in ai_messages:
            print(ai_message.content)  # Print the content of each AI message
    # Check if the chunk contains tool messages
    if 'tools' in chunk:
        tool_messages = chunk['tools']['messages']
        print("Tool Messages:")
        for tool_message in tool_messages:
            print(tool_message.content)  # Print the content of each ToolMessage



Loaded .env from c:\Users\rickcau\source\repos\LangGraph-101\.env
API Key: 8PVz****************************Isv1
Streaming response for the detailed analysis:
AI Messages:

Tool Messages:
Confidentiality Analysis:
- Scope: All information shared during the project shall re...
- Duration: Searching for time specifications...
- Obligations: Identifying key responsibilities...
Termination Analysis:
- Notice Period: Searching...
- Grounds for Termination: Analyzing conditions...
- Post-termination obligations: Reviewing...
Liability Analysis:
- Limitation Scope: Total liability shall not exceed the total contrac...
- Cap Amount: Searching for monetary limits...
- Exclusions: Identifying carve-outs...
AI Messages:
Here are the detailed analyses for each of the specified contract clauses:

1. **Confidentiality Clause**:
   - **Scope**: The clause specifies that all information shared during the project is to be kept confidential. This broad scope suggests that any data, documents, or communic